# FLOPs

In [53]:
import paddle
# https://github.com/Lyken17/pytorch-OpCounter/tree/master/thop
# https://arxiv.org/pdf/1611.06440.pdf

In [54]:
model = paddle.vision.models.resnet34()

In [55]:
paddle.summary(model, (1, 3, 224, 224))

-------------------------------------------------------------------------------
   Layer (type)         Input Shape          Output Shape         Param #    
    Conv2D-217       [[1, 3, 224, 224]]   [1, 64, 112, 112]        9,408     
  BatchNorm2D-217   [[1, 64, 112, 112]]   [1, 64, 112, 112]         256      
     ReLU-103       [[1, 64, 112, 112]]   [1, 64, 112, 112]          0       
    MaxPool2D-7     [[1, 64, 112, 112]]    [1, 64, 56, 56]           0       
    Conv2D-218       [[1, 64, 56, 56]]     [1, 64, 56, 56]        36,864     
  BatchNorm2D-218    [[1, 64, 56, 56]]     [1, 64, 56, 56]          256      
     ReLU-104        [[1, 64, 56, 56]]     [1, 64, 56, 56]           0       
    Conv2D-219       [[1, 64, 56, 56]]     [1, 64, 56, 56]        36,864     
  BatchNorm2D-219    [[1, 64, 56, 56]]     [1, 64, 56, 56]          256      
   BasicBlock-97     [[1, 64, 56, 56]]     [1, 64, 56, 56]           0       
    Conv2D-220       [[1, 64, 56, 56]]     [1, 64, 56, 56]    

{'total_params': 21814696, 'trainable_params': 21780648}

In [56]:
totle_num = 0

def _conv2d_hook(layer, inputs, output):
    '''
    '''
    o = output.shape[0] * output.shape[2] * output.shape[2]   
    k = layer.weight.numel() 
    b = 1 if layer.bias is not None else 0
    
    layer.op_num += o * (k + b)


def _relu_hook(layer, inputs, output):
    '''
    '''
    layer.op_num += inputs[0].numel()
    

def _linear_hook(layer, inputs, output):
    '''
    '''
    in_channels = layer.weight.shape[0]    
    layer.op_num += in_channels * output.numel()


hooks = {
    paddle.nn.Conv2D : _conv2d_hook,
    paddle.nn.ReLU : _relu_hook,
    paddle.nn.Linear : _linear_hook,
}



def register_hook(model):
    
    for m in model.sublayers():
        if type(m) in hooks:
            m.register_forward_post_hook( hooks[type(m)] )
            m.register_buffer('op_num', paddle.zeros(shape=(1, ), dtype='float32'))
            
    print('register done...')

In [57]:
register_hook(model)

register done...


In [58]:
model.eval()
model(paddle.rand((1, 3, 224, 224))).shape

[1, 1000]

In [59]:
for i, m in enumerate(model.sublayers()):
    if hasattr(m, 'op_num'):
        print(i, m, m.op_num.numpy()[0])

0 Conv2D(3, 64, kernel_size=[7, 7], stride=[2, 2], padding=3, data_format=NCHW) 118013950.0
2 ReLU() 802816.0
6 Conv2D(64, 64, kernel_size=[3, 3], padding=1, data_format=NCHW) 115605500.0
8 ReLU() 401408.0
9 Conv2D(64, 64, kernel_size=[3, 3], padding=1, data_format=NCHW) 115605500.0
12 Conv2D(64, 64, kernel_size=[3, 3], padding=1, data_format=NCHW) 115605500.0
14 ReLU() 401408.0
15 Conv2D(64, 64, kernel_size=[3, 3], padding=1, data_format=NCHW) 115605500.0
18 Conv2D(64, 64, kernel_size=[3, 3], padding=1, data_format=NCHW) 115605500.0
20 ReLU() 401408.0
21 Conv2D(64, 64, kernel_size=[3, 3], padding=1, data_format=NCHW) 115605500.0
25 Conv2D(64, 128, kernel_size=[3, 3], stride=[2, 2], padding=1, data_format=NCHW) 57802750.0
27 ReLU() 200704.0
28 Conv2D(128, 128, kernel_size=[3, 3], padding=1, data_format=NCHW) 115605500.0
31 Conv2D(64, 128, kernel_size=[1, 1], stride=[2, 2], data_format=NCHW) 6422528.0
34 Conv2D(128, 128, kernel_size=[3, 3], padding=1, data_format=NCHW) 115605500.0
36 Re